1. 문서 내용을 읽는다
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tenacity import wait_chain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 청크 하나
    chunk_overlap=200, # 자를때 딱딱 나눠서 자르는게 아니라 어느정도 겹치게 잘라주는 것
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model='text-embedding-3-large') # 디폴트 모델이 별로라 성능 좋은 모델 넣어준

In [ ]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax',persist_directory="./chroma")

database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)

In [ ]:
query = '연봉 5천만원의 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

In [ ]:
# prompt = f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가 입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요
#
# [Context]
# {retrieved_docs}
#
# Question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [ ]:
# ai_message.content

In [ ]:
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message